<a href="https://colab.research.google.com/github/donghuna/PromptGenerate/blob/main/generateCode-codeLlama-myAgent.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install datasets
!pip install langchain
!pip install tiktoken
!pip install langchain-openai
!pip install faiss-cpu
!pip install langchain-community
!pip install sentence_transformers
!pip install peft
# !pip install bitsandbytes
!pip install -U bitsandbytes
!pip install --upgrade torch
!pip install transformers
!pip install huggingface_hub
!pip install accelerate

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 10.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 12.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 17.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 13.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 20.4 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.10.0:
      Successfully uninstalled fsspec-2024.10.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 13.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.4/50.4 kB 2.2 MB/s eta 0:00:00
   ━━

In [2]:
from datetime import datetime
import transformers
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline, BitsAndBytesConfig
from datasets import load_dataset
import csv

from peft import prepare_model_for_kbit_training, LoraConfig, PeftModel, get_peft_model
import pandas as pd

In [3]:
from google.colab import userdata

from huggingface_hub import login
login(token=userdata.get('HUG_TOKEN'))

The token has not been saved to the git credentials helper. Pass `add_to_git_credential=True` in this function directly or `--add-to-git-credential` if using via `huggingface-cli` if you want to set the git credential as well.
Token is valid (permission: fineGrained).
Your token has been saved to /root/.cache/huggingface/token
Login successful


In [7]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


# Retrieve agent

In [5]:
from transformers import AutoModelForCausalLM, AutoTokenizer
from sentence_transformers import SentenceTransformer

# 모델 경로
embedding_model_path = "sentence-transformers/paraphrase-MiniLM-L6-v2"

# GPU 확인
device = "cuda" if torch.cuda.is_available() else "cpu"

# 모델 로드
embedding_model = SentenceTransformer(embedding_model_path, device=device)

embedding_model.eval()

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/229 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/122 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/3.73k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

SentenceTransformer(
  (0): Transformer({'max_seq_length': 128, 'do_lower_case': False}) with Transformer model: BertModel 
  (1): Pooling({'word_embedding_dimension': 384, 'pooling_mode_cls_token': False, 'pooling_mode_mean_tokens': True, 'pooling_mode_max_tokens': False, 'pooling_mode_mean_sqrt_len_tokens': False, 'pooling_mode_weightedmean_tokens': False, 'pooling_mode_lasttoken': False, 'include_prompt': True})
)

In [8]:
import faiss

## FAISS index
index = faiss.read_index("/content/drive/MyDrive/promptGenerate/question_faiss_index-sentence.faiss")

rag_dataset = load_dataset("koutch/staqc", 'sca_python')
rag_dataset

README.md:   0%|          | 0.00/8.78k [00:00<?, ?B/s]

staqc.py:   0%|          | 0.00/9.62k [00:00<?, ?B/s]

0000.parquet:   0%|          | 0.00/22.6M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/85294 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['question_id', 'question', 'snippet'],
        num_rows: 85294
    })
})

In [9]:
import numpy as np

def normalize(vectors):
    norms = np.linalg.norm(vectors, axis=1, keepdims=True)
    return vectors / norms

# 유사도 검색 함수
def RAG_AGENT(query_sentence, k=1):
    query_embedding = embedding_model.encode([query_sentence])
    query_embedding_normalized = normalize(query_embedding)

    # FAISS 인덱스에서 가장 유사한 벡터 k개 검색
    distances, indices = index.search(np.array([query_embedding_normalized[0]], dtype='float32'), k)

    # 검색된 인덱스와 유사도를 반환
    similar_questions = []
    for i in range(k):
        question = rag_dataset['train']['question'][indices[0][i]]
        snippet = rag_dataset['train']['snippet'][indices[0][i]]
        distance = distances[0][i]

        # print(f"원래 질문: {query_sentence}, 문서 id: {indices[0][i]}, 유사도: {distances[0][i]}, 질문: {question}")
        # similar_questions.append((question, snippet, distance))

    return question, snippet, distance

In [10]:
dataset = load_dataset("openai_humaneval")
dataset

README.md:   0%|          | 0.00/6.52k [00:00<?, ?B/s]

test-00000-of-00001.parquet:   0%|          | 0.00/83.9k [00:00<?, ?B/s]

Generating test split:   0%|          | 0/164 [00:00<?, ? examples/s]

DatasetDict({
    test: Dataset({
        features: ['task_id', 'prompt', 'canonical_solution', 'test', 'entry_point'],
        num_rows: 164
    })
})

# Planning Agent

In [11]:
from transformers import AutoTokenizer, AutoModelForCausalLM
from peft import PeftModel
import torch

tokenizer = AutoTokenizer.from_pretrained("donghuna/llama-plan-1B")
base_model = AutoModelForCausalLM.from_pretrained("meta-llama/Llama-3.2-1B-Instruct", return_dict=True)
peft_model_id = "donghuna/llama-plan-1B"
plan_model = PeftModel.from_pretrained(base_model, peft_model_id).to("cuda")
plan_model.eval()


tokenizer_config.json:   0%|          | 0.00/54.6k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/325 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/877 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/2.47G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/189 [00:00<?, ?B/s]

adapter_config.json:   0%|          | 0.00/749 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/peft/tuners/tuners_utils.py:500: UserWarning: Model with `tie_word_embeddings=True` and the tied_target_modules=['lm_head'] are part of the adapter. This can lead to complications, for example when merging the adapter or converting your model to formats other than safetensors. See for example https://github.com/huggingface/peft/issues/2018.
  warnings.warn(


adapter_model.safetensors:   0%|          | 0.00/1.08G [00:00<?, ?B/s]

PeftModelForCausalLM(
  (base_model): LoraModel(
    (model): LlamaForCausalLM(
      (model): LlamaModel(
        (embed_tokens): Embedding(128256, 2048)
        (layers): ModuleList(
          (0-15): 16 x LlamaDecoderLayer(
            (self_attn): LlamaSdpaAttention(
              (q_proj): lora.Linear(
                (base_layer): Linear(in_features=2048, out_features=2048, bias=False)
                (lora_dropout): ModuleDict(
                  (default): Dropout(p=0.05, inplace=False)
                )
                (lora_A): ModuleDict(
                  (default): Linear(in_features=2048, out_features=8, bias=False)
                )
                (lora_B): ModuleDict(
                  (default): Linear(in_features=8, out_features=2048, bias=False)
                )
                (lora_embedding_A): ParameterDict()
                (lora_embedding_B): ParameterDict()
                (lora_magnitude_vector): ModuleDict()
              )
              (k_proj): lora.Line

In [12]:


def generate_and_tokenize_planning_prompt(original_question, similar_question, snippet, confidence):
    full_prompt = f"""
<|begin_of_text|><|start_header_id|>system<|end_header_id|>
Given a original competitive programming problem, generate a concrete planning to solve the problem.
The provided retrieved question and code are examples to help generate the plan.<|eot_id|>
<|start_header_id|>user<|end_header_id|>
### Original Problem to be solved :
{original_question}

### Retrieved Problem (similar question) :
{similar_question}

### Retrieved code snippet (related to the retrieved question) :
{snippet}

### Similarity score (between original problem and retrieved question) :
{confidence}
--------------------
Important: You should give only the planning to solve the problem. Do not add extra explanation or words or code.
Let's think step by step.

### Planning to solve the problem:<|eot_id|>
<|start_header_id|>assistant<|end_header_id|>
"""
    return full_prompt

terminators = [
    tokenizer.eos_token_id,
    tokenizer.convert_tokens_to_ids("<|eot_id|>")
]

def PLANNING_AGENT(original_question, similar_question, snippet, confidence):
    prompt = generate_and_tokenize_planning_prompt(original_question, similar_question, snippet, confidence)
    inputs = tokenizer(prompt, return_tensors="pt", max_length=512, truncation=True).to(device)

    # 입력 토큰의 길이 계산
    input_length = inputs['input_ids'].shape[1]

    # 입력 프롬프트 토큰 수에 128을 더한 max_length 설정
    max_length = input_length + 256

    with torch.no_grad():
        output_ids = plan_model.generate(
            **inputs,
            max_length=max_length,
            eos_token_id=terminators,
            pad_token_id=tokenizer.eos_token_id,
            temperature=0.2
        )

        generated_text = tokenizer.decode(output_ids[0], skip_special_tokens=False)

        # print(generated_text)
        generated_plan = ""

        keyword = "<|start_header_id|>assistant<|end_header_id|>"
        if keyword in generated_text:
            generated_plan = generated_text.split(keyword)[1].strip()

        keyword = "<|eot_id|>"
        if keyword in generated_plan:
            generated_plan = generated_plan.split(keyword)[0].strip()

        # if not generated_plan:
        #     continue

        # print(generated_plan)
        return generated_plan

# Code generator

In [13]:
# code_model = "codellama/CodeLlama-7b-hf"
code_model = "codellama/CodeLlama-7b-Instruct-hf"

code_tokenizer = AutoTokenizer.from_pretrained(code_model, truncation=True)
pipeline = transformers.pipeline(
    "text-generation",
    model=code_model,
    tokenizer=code_tokenizer,
    torch_dtype=torch.float16,
    device_map="auto",
)

tokenizer_config.json:   0%|          | 0.00/1.59k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.84M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/411 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/646 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/25.1k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/9.98G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/3.50G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

In [14]:
# system_prompt = """You are an AI specialized in generating Python code. Only produce Python code that directly corresponds to the provided comments or function descriptions without any additional explanations or extraneous text."""
system_prompt = "You are an AI assistant that generates Python code based on provided instructions."


def generate_and_tokenize_prompt_for_generate_code(original_question, similar_question, snippet, confidence, plan):
    full_prompt = f"""
[INST] <<SYS>>{system_prompt}<</SYS>>
# Below is a Python code that requires implementation. Please complete the parts indicated by comments.

# Initial code and sections that need implementation:
{original_question}

# Implementation plan:
{plan}

# Based on the above information, complete the commented sections.
[/INST]
"""

    """

## Problem to be solved :
{original_question}
## Planning :
{plan}

    Given a competitive programming problem generate Python3 code to solve the problem.


## Let's think step by step.
------------
Important:
## Your response must contain only the Python code to solve this problem. Do not add extra explanation or words.
"""
    return full_prompt

# Execute pipeline

In [ ]:
output_file = "generated_code-withAgentPrompt.csv"

with open(output_file, mode="w", newline="", encoding="utf-8") as file:
    writer = csv.writer(file)
    writer.writerow(["task_id", "prompt", "solution", "test", "entry_point"])  # CSV 헤더 작성

    for idx, data in enumerate(dataset["test"]):
        task_id = data["task_id"]
        data_prompt = data["prompt"]
        test = data["test"]
        entry_point = data["entry_point"]

        similar_question, similar_snippet, distance = RAG_AGENT(data_prompt)
        # print(similar_question, similar_snippet)

        plan = PLANNING_AGENT(data_prompt, similar_question, similar_snippet, distance)
        # print(plan)

        prompt_for_generate_code = generate_and_tokenize_prompt_for_generate_code(data_prompt, similar_question, similar_snippet, distance, plan)
        # print(prompt_for_generate_code)

        # 입력 프롬프트를 토크나이즈하여 길이 계산
        input_ids = code_tokenizer.encode(data_prompt, return_tensors='pt', truncation=True, max_length=512).to('cuda')
        input_length = input_ids.shape[-1]

        # max_new_tokens를 입력 프롬프트 길이 + 100으로 설정
        max_new_tokens = input_length + 400

        # print(prompt_for_generate_code)
        # break

        sequences = pipeline(
            prompt_for_generate_code,
            # data_prompt,
            do_sample=True,
            temperature=0.2,
            top_p=0.9,
            num_return_sequences=1,
            eos_token_id=code_tokenizer.eos_token_id,
            pad_token_id=code_tokenizer.eos_token_id,
            repetition_penalty=1.5,
            max_new_tokens=max_new_tokens,
            # max_length=500,
        )

        generated_code = sequences[0]['generated_text']
        print(sequences[0]['generated_text'])

        writer.writerow([task_id, data_prompt, generated_code, test, entry_point])
        # writer.writerow([task_id, data_prompt.replace('\n', '\\n'), generated_code.replace('\n', '\\n')])
        # break


[INST] <<SYS>>You are an AI assistant that generates Python code based on provided instructions.<</SYS>>
# Below is a Python code that requires implementation. Please complete the parts indicated by comments.

# Initial code and sections that need implementation:
from typing import List


def has_close_elements(numbers: List[float], threshold: float) -> bool:
    """ Check if in given list of numbers, are any two numbers closer to each other than
    given threshold.
    >>> has_close_elements([1.0, 2.0, 3.0], 0.5)
    False
    >>> has_close_elements([1.0, 2.8, 3.0, 4.0, 5.0, 2.0], 0.3)
    True
    """


# Implementation plan:
1. Define a function to check if two numbers are equal.
2. Define a function to check if two numbers are within a certain threshold of each other.
3. Use the two functions to check if any two numbers in the list are within the threshold of each other.
4. Return True if any two numbers are within the threshold of each other, False otherwise.

# Based on the abo

You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset



[INST] <<SYS>>You are an AI assistant that generates Python code based on provided instructions.<</SYS>>
# Below is a Python code that requires implementation. Please complete the parts indicated by comments.

# Initial code and sections that need implementation:
from typing import List, Tuple


def sum_product(numbers: List[int]) -> Tuple[int, int]:
    """ For a given list of integers, return a tuple consisting of a sum and a product of all the integers in a list.
    Empty sum should be equal to 0 and empty product should be equal to 1.
    >>> sum_product([])
    (0, 1)
    >>> sum_product([1, 2, 3, 4])
    (10, 24)
    """


# Implementation plan:
1. Define a function that takes a list of integers as input and returns a tuple of sum and product of all the integers in the list.
2. Use the built-in `sum` function to calculate the sum of all integers in the list.
3. Use the `map` function to apply the `operator.add` function to each integer in the list, effectively calculating the p

In [ ]:
# !cp generated_code-agent.csv /content/drive/MyDrive/

import pandas as pd

df = pd.read_csv(output_file)

from datasets import Dataset
from google.colab import userdata

dataset = Dataset.from_pandas(df)
dataset.push_to_hub("donghuna/generated_code-Agent", token=userdata.get('HUG_TOKEN')